<a href="https://colab.research.google.com/github/tejvirus0003/mypynb/blob/main/random_forest_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

Problem Statement In the lending industry, investors provide loans to borrowers in exchange for the promise of repayment with interest. If the borrower repays the loan, then the lender profits from the interest. However, if the borrower is unable to repay the loan, then the lender loses money. Therefore, lenders face the problem of predicting the risk of a borrower being unable to repay a loan. Your task is to predict whether a person should be given the loan or not using the publicly available Link to the dataset https://www.kaggle.com/wordsforthewise/lending-club

Perform the pre-processing operations as and when required.
Divide the data into training and testing parts.
Compare the performance of the Random Forest and Decision Trees classifier as to which is the better option to solve the above problem statement.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/accepted_2007_to_2018Q4.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,...,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,...,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,home_improvement,NaN,605xx,IL,10.78,0.0,Aug-2000,695.0,699.0,0.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,0.00,0.00,...,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0.0,Sep-2008,785.0,789.0,0.0,NaN,NaN,13.0,0.0,7802.0,11.6,17.0,w,15897.65,15897.65,...,381215.0,52226.0,62500.0,18000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,major_purchase,Major purchase,174xx,PA,25.37,1.0,Jun-1998,695.0,699.0,3.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,0.00,0.00,...,439570.0,95768.0,20300.0,88097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sup = data.iloc[0,:]

In [6]:
dict(sup.T)

{'acc_now_delinq': 0.0,
 'acc_open_past_24mths': 4.0,
 'addr_state': 'PA',
 'all_util': 34.0,
 'annual_inc': 55000.0,
 'annual_inc_joint': nan,
 'application_type': 'Individual',
 'avg_cur_bal': 20701.0,
 'bc_open_to_buy': 1506.0,
 'bc_util': 37.2,
 'chargeoff_within_12_mths': 0.0,
 'collection_recovery_fee': 0.0,
 'collections_12_mths_ex_med': 0.0,
 'debt_settlement_flag': 'N',
 'debt_settlement_flag_date': nan,
 'deferral_term': nan,
 'delinq_2yrs': 0.0,
 'delinq_amnt': 0.0,
 'desc': nan,
 'disbursement_method': 'Cash',
 'dti': 5.91,
 'dti_joint': nan,
 'earliest_cr_line': 'Aug-2003',
 'emp_length': '10+ years',
 'emp_title': 'leadman',
 'fico_range_high': 679.0,
 'fico_range_low': 675.0,
 'funded_amnt': 3600.0,
 'funded_amnt_inv': 3600.0,
 'grade': 'C',
 'hardship_amount': nan,
 'hardship_dpd': nan,
 'hardship_end_date': nan,
 'hardship_flag': 'N',
 'hardship_last_payment_amount': nan,
 'hardship_length': nan,
 'hardship_loan_status': nan,
 'hardship_payoff_balance_amount': nan,
 'h

In [7]:
hi = dict(data.isnull().sum())

In [8]:
hi

{'acc_now_delinq': 62,
 'acc_open_past_24mths': 50063,
 'addr_state': 33,
 'all_util': 866381,
 'annual_inc': 37,
 'annual_inc_joint': 2139991,
 'application_type': 33,
 'avg_cur_bal': 70379,
 'bc_open_to_buy': 74968,
 'bc_util': 76104,
 'chargeoff_within_12_mths': 178,
 'collection_recovery_fee': 33,
 'collections_12_mths_ex_med': 178,
 'debt_settlement_flag': 33,
 'debt_settlement_flag_date': 2226455,
 'deferral_term': 2249784,
 'delinq_2yrs': 62,
 'delinq_amnt': 62,
 'desc': 2134634,
 'disbursement_method': 33,
 'dti': 1744,
 'dti_joint': 2139995,
 'earliest_cr_line': 62,
 'emp_length': 146940,
 'emp_title': 167002,
 'fico_range_high': 33,
 'fico_range_low': 33,
 'funded_amnt': 33,
 'funded_amnt_inv': 33,
 'grade': 33,
 'hardship_amount': 2249784,
 'hardship_dpd': 2249784,
 'hardship_end_date': 2249784,
 'hardship_flag': 33,
 'hardship_last_payment_amount': 2249784,
 'hardship_length': 2249784,
 'hardship_loan_status': 2249784,
 'hardship_payoff_balance_amount': 2249784,
 'hardship_

In [9]:
tohave = ['loan_amnt' , 'term' , 'int_rate' , 'installment' , 'grade' , 'sub_grade' , 'emp_title' , 'emp_length' , 'home_ownership' , 'annual_inc' ,'verification_status','issue_d','loan_status','purpose','title' , 'zip_code','addr_state' , 'dti' , 'earliest_cr_line' , 'open_acc' , 'pub_rec' , 'revol_bal','revol_util','total_acc','initial_list_status' , 'application_type' ,'mort_acc' , 'pub_rec_bankruptcies' ]

In [10]:
#0	loan_amnt	The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
#1	term	The number of payments on the loan. Values are in months and can be either 36 or 60.
#2	int_rate	Interest Rate on the loan
#3	installment	The monthly payment owed by the borrower if the loan originates.
#4	grade	LC assigned loan grade
#5	sub_grade	LC assigned loan subgrade
#6	emp_title	The job title supplied by the Borrower when applying for the loan.*
#7	emp_length	Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
#8	home_ownership	The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
#9	annual_inc	The self-reported annual income provided by the borrower during registration.
#10	verification_status	Indicates if income was verified by LC, not verified, or if the income source was verified
#11	issue_d	The month which the loan was funded
#12	loan_status	Current status of the loan
#13	purpose	A category provided by the borrower for the loan request.
#14	title	The loan title provided by the borrower
#15	zip_code	The first 3 numbers of the zip code provided by the borrower in the loan application.
#16	addr_state	The state provided by the borrower in the loan application
#17	dti	A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
#18	earliest_cr_line	The month the borrower's earliest reported credit line was opened
#19	open_acc	The number of open credit lines in the borrower's credit file.
#20	pub_rec	Number of derogatory public records
#21	revol_bal	Total credit revolving balance
#22	revol_util	Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
#23	total_acc	The total number of credit lines currently in the borrower's credit file
#24	initial_list_status	The initial listing status of the loan. Possible values are – W, F
#25	application_type	Indicates whether the loan is an individual application or a joint application with two co-borrowers
#26	mort_acc	Number of mortgage accounts.
#27	pub_rec_bankruptcies	Number of public record 

In [11]:
to_remove = []
for i in data.columns :
    if i not in tohave :
        to_remove.append(i)

In [12]:
data.drop(to_remove , axis=1 , inplace=True)

In [13]:
data.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation,190xx,PA,5.91,Aug-2003,7.0,0.0,2765.0,29.7,13.0,w,Individual,1.0,0.0
1,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,Business,577xx,SD,16.06,Dec-1999,22.0,0.0,21470.0,19.2,38.0,w,Individual,4.0,0.0
2,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,NaN,605xx,IL,10.78,Aug-2000,6.0,0.0,7869.0,56.2,18.0,w,Joint App,5.0,0.0
3,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,Debt consolidation,076xx,NJ,17.06,Sep-2008,13.0,0.0,7802.0,11.6,17.0,w,Individual,1.0,0.0
4,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,Major purchase,174xx,PA,25.37,Jun-1998,12.0,0.0,21929.0,64.5,35.0,w,Individual,6.0,0.0


In [14]:
data.isnull().sum()

loan_amnt                   33
term                        33
int_rate                    33
installment                 33
grade                       33
sub_grade                   33
emp_title               167002
emp_length              146940
home_ownership              33
annual_inc                  37
verification_status         33
issue_d                     33
loan_status                 33
purpose                     33
title                    23358
zip_code                    34
addr_state                  33
dti                       1744
earliest_cr_line            62
open_acc                    62
pub_rec                     62
revol_bal                   33
revol_util                1835
total_acc                   62
initial_list_status         33
application_type            33
mort_acc                 50063
pub_rec_bankruptcies      1398
dtype: int64

In [15]:
data.dropna(inplace=True)

In [16]:
data['term'] = data['term'].apply(lambda x : int(x.split()[0]) )

In [17]:
len(data)

2024455

In [18]:
data.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation,190xx,PA,5.91,Aug-2003,7.0,0.0,2765.0,29.7,13.0,w,Individual,1.0,0.0
1,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,Business,577xx,SD,16.06,Dec-1999,22.0,0.0,21470.0,19.2,38.0,w,Individual,4.0,0.0
3,35000.0,60,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,Debt consolidation,076xx,NJ,17.06,Sep-2008,13.0,0.0,7802.0,11.6,17.0,w,Individual,1.0,0.0
4,10400.0,60,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,Major purchase,174xx,PA,25.37,Jun-1998,12.0,0.0,21929.0,64.5,35.0,w,Individual,6.0,0.0
5,11950.0,36,13.44,405.18,C,C3,Veterinary Tecnician,4 years,RENT,34000.0,Source Verified,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation,300xx,GA,10.20,Oct-1987,5.0,0.0,8822.0,68.4,6.0,w,Individual,0.0,0.0


In [19]:
data.dtypes

loan_amnt               float64
term                      int64
int_rate                float64
installment             float64
grade                    object
sub_grade                object
emp_title                object
emp_length               object
home_ownership           object
annual_inc              float64
verification_status      object
issue_d                  object
loan_status              object
purpose                  object
title                    object
zip_code                 object
addr_state               object
dti                     float64
earliest_cr_line         object
open_acc                float64
pub_rec                 float64
revol_bal               float64
revol_util              float64
total_acc               float64
initial_list_status      object
application_type         object
mort_acc                float64
pub_rec_bankruptcies    float64
dtype: object

In [20]:
enc1 = pd.get_dummies(data['grade'] , drop_first=True)

In [21]:
data.join(enc1)

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,B,C,D,E,F,G
0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation,190xx,PA,5.91,Aug-2003,7.0,0.0,2765.0,29.7,13.0,w,Individual,1.0,0.0,0,1,0,0,0,0
1,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,Business,577xx,SD,16.06,Dec-1999,22.0,0.0,21470.0,19.2,38.0,w,Individual,4.0,0.0,0,1,0,0,0,0
3,35000.0,60,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,Debt consolidation,076xx,NJ,17.06,Sep-2008,13.0,0.0,7802.0,11.6,17.0,w,Individual,1.0,0.0,0,1,0,0,0,0
4,10400.0,60,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,Major purchase,174xx,PA,25.37,Jun-1998,12.0,0.0,21929.0,64.5,35.0,w,Individual,6.0,0.0,0,0,0,0,1,0
5,11950.0,36,13.44,405.18,C,C3,Veterinary Tecnician,4 years,RENT,34000.0,Source Verified,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation,300xx,GA,10.20,Oct-1987,5.0,0.0,8822.0,68.4,6.0,w,Individual,0.0,0.0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260692,16000.0,60,12.79,362.34,C,C1,Manager,10+ years,RENT,150000.0,Not Verified,Oct-2016,Fully Paid,medical,Medical expenses,284xx,NC,12.25,Aug-1997,12.0,4.0,7700.0,55.0,28.0,f,Individual,0.0,3.0,0,1,0,0,0,0
2260693,24000.0,60,10.49,515.74,B,B3,Current Operations Officer,4 years,OWN,125000.0,Not Verified,Oct-2016,Current,credit_card,Credit card refinancing,967xx,HI,10.98,Feb-2001,15.0,2.0,22448.0,22.4,22.0,f,Individual,0.0,0.0,1,0,0,0,0,0
2260694,24000.0,60,12.79,543.50,C,C1,Unit Operator,7 years,MORTGAGE,95000.0,Source Verified,Oct-2016,Current,home_improvement,Home improvement,356xx,AL,19.61,Dec-1999,5.0,0.0,49431.0,84.4,54.0,f,Individual,0.0,0.0,0,1,0,0,0,0
2260695,24000.0,60,10.49,515.74,B,B3,Database Administrator,10+ years,MORTGAGE,108000.0,Not Verified,Oct-2016,Current,debt_consolidation,Debt consolidation,840xx,UT,34.94,Feb-1991,24.0,1.0,21665.0,39.0,58.0,f,Individual,4.0,0.0,1,0,0,0,0,0


In [22]:
for i in data.columns.to_list() :
  print(i , len(data[i].unique()))

loan_amnt 1561
term 2
int_rate 366
installment 89643
grade 7
sub_grade 35
emp_title 481881
emp_length 11
home_ownership 6
annual_inc 76388
verification_status 3
issue_d 82
loan_status 7
purpose 14
title 41395
zip_code 952
addr_state 51
dti 9334
earliest_cr_line 734
open_acc 90
pub_rec 42
revol_bal 99377
revol_util 1328
total_acc 150
initial_list_status 2
application_type 2
mort_acc 47
pub_rec_bankruptcies 12


In [23]:
to_make_dummy = [ 'grade' , 'sub_grade' , 'emp_length' , 'home_ownership' , 'verification_status' , 'loan_status' , 'initial_list_status' , 'application_type']
to_drop = ['emp_title' , 'issue_d' , 'purpose' ,'title' , 'zip_code' ,'addr_state' , 'earliest_cr_line' ]

In [24]:
data['emp_length'].unique()

array(['10+ years', '3 years', '4 years', '6 years', '1 year', '7 years',
       '8 years', '5 years', '2 years', '9 years', '< 1 year'],
      dtype=object)

In [25]:
data.drop(to_drop , axis=1 , inplace=True)

In [26]:
data.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,3600.0,36,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,Fully Paid,5.91,7.0,0.0,2765.0,29.7,13.0,w,Individual,1.0,0.0
1,24700.0,36,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,Fully Paid,16.06,22.0,0.0,21470.0,19.2,38.0,w,Individual,4.0,0.0
3,35000.0,60,14.85,829.90,C,C5,10+ years,MORTGAGE,110000.0,Source Verified,Current,17.06,13.0,0.0,7802.0,11.6,17.0,w,Individual,1.0,0.0
4,10400.0,60,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,Source Verified,Fully Paid,25.37,12.0,0.0,21929.0,64.5,35.0,w,Individual,6.0,0.0
5,11950.0,36,13.44,405.18,C,C3,4 years,RENT,34000.0,Source Verified,Fully Paid,10.20,5.0,0.0,8822.0,68.4,6.0,w,Individual,0.0,0.0


In [27]:
data.to_csv('removed_na_loan.csv')